# Basic Imports

In [ ]:
from google.colab import drive
import sys
import os
import pandas as pd
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


# Data extraction of the valid cfp and wfp

In [ ]:
co2_sueatable = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2.csv", sep= ';')
co2_sueatable['final_co2'] = None

wfp_sueatable = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp.csv", sep= ';')
wfp_sueatable['final_wfp'] = None

In [ ]:
co2_sueatable['final_co2'] = np.where(co2_sueatable['Suggested CF value'].eq('OK item'), co2_sueatable['Carbon Footprint kg CO2eq/kg or l of food ITEM'], co2_sueatable['final_co2'])
co2_sueatable['final_co2'] = np.where(co2_sueatable['Suggested CF value'].eq('Item matching typology'), co2_sueatable['Carbon Footprint kg CO2eq/kg or l of food ITEM'], co2_sueatable['final_co2'])
co2_sueatable['final_co2'] = np.where(co2_sueatable['Suggested CF value'].eq('item or typology'), co2_sueatable['Carbon Footprint kg CO2eq/kg or l of food ITEM'], co2_sueatable['final_co2'])
co2_sueatable['final_co2'] = np.where(co2_sueatable['Suggested CF value'].eq('item or typology or subtypology'), co2_sueatable['Carbon Footprint kg CO2eq/kg or l of food ITEM'], co2_sueatable['final_co2'])
co2_sueatable['final_co2'] = np.where(co2_sueatable['Suggested CF value'].eq('better typol. or subtypol.'), co2_sueatable['Carbon Footprint g CO2eq/g o cc of food sub-TYPOLOGY'], co2_sueatable['final_co2'])
co2_sueatable['final_co2'] = np.where(co2_sueatable['Suggested CF value'].eq('better typology'), co2_sueatable['Carbon Footprint g CO2eq/g o cc of food TYPOLOGY'], co2_sueatable['final_co2'])
co2_sueatable['final_co2'] = np.where(co2_sueatable['Suggested CF value'].eq('better subtypology'), co2_sueatable['Carbon Footprint g CO2eq/g o cc of food sub-TYPOLOGY'], co2_sueatable['final_co2'])


In [ ]:
wfp_sueatable['final_wfp'] = np.where(wfp_sueatable['Suggested WF value'].eq('ok item'), wfp_sueatable['Water Footprint liters water/kg o liter of food ITEM'], wfp_sueatable['final_wfp'])
wfp_sueatable['final_wfp'] = np.where(wfp_sueatable['Suggested WF value'].eq('item matching typology'), wfp_sueatable['Water Footprint liters water/kg o liter of food ITEM'], wfp_sueatable['final_wfp'])
wfp_sueatable['final_wfp'] = np.where(wfp_sueatable['Suggested WF value'].eq('item or typology'), wfp_sueatable['Water Footprint liters water/kg o liter of food ITEM'], wfp_sueatable['final_wfp'])
wfp_sueatable['final_wfp'] = np.where(wfp_sueatable['Suggested WF value'].eq('better typology'), wfp_sueatable['Water Footprint cc water/g o cc of food TYPOLOGY'], wfp_sueatable['final_wfp'])
wfp_sueatable['final_wfp'] = np.where(wfp_sueatable['Suggested WF value'].eq('better sub-typology'), wfp_sueatable['Water Footprint cc water/g o cc of food sub-TYPOLOGY'], wfp_sueatable['final_wfp'])


In [ ]:
columns = ['FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2']
co2_sueatable[columns].to_csv('/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2_filtered.csv', sep= ';')

In [ ]:
columns = ['FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_wfp']
wfp_sueatable[columns].to_csv('/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp_filtered.csv', sep= ';')

# Join of the two dataframe in a single one with both the informations
Now I want to bind, **for each ingredients of the co2 footprint** the relative information about the water footprint.

This is realized using different steps:

1.   Matching of the same string.
2.   Matching of the simplified strings removing (I) and *.
3.   For all the non covered data I will use the the approach suggested by the authors, using the value of the relative subtypology (if avalaible) or the typology.
4.   Saving the results



### Imports and initialization

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
co2_sueat = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2_filtered.csv", sep= ';')
wfp_sueat = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp_filtered.csv", sep= ';')
co2_sueat['final_wfp'] = None

###Step 1:

Matching of the same string.

In [ ]:
#not useful since the step 2 perform also this one
#updated = pd.merge(co2_sueat, wfp_sueat, how='left', on = 'Food commodity ITEM', suffixes=('', '_new'))
#updated['final_wfp'] = np.where(pd.notnull(updated['final_wfp_new']), updated['final_wfp_new'], updated['final_wfp'])
#updated.drop('Value_new', axis=1, inplace=True)
#columns = ['FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
#co2_sueat = updated[columns]
#co2_sueat[pd.notnull(co2_sueat['final_wfp'])]
#127 updated

###Step 2:

Matching of the simplified strings removing (I), (g), (G) and * both sides.

In [ ]:
co2_sueat['Food commodity ITEM'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].map(lambda x: x.strip()), co2_sueat['Food commodity ITEM'])
co2_sueat['Food commodity ITEM'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].map(lambda x: x.replace('*','')), co2_sueat['Food commodity ITEM'])
co2_sueat['Food commodity ITEM'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].map(lambda x: x.replace(' (I)','')), co2_sueat['Food commodity ITEM'])
co2_sueat['Food commodity ITEM'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].map(lambda x: x.replace(' (g)','')), co2_sueat['Food commodity ITEM'])
co2_sueat['Food commodity ITEM'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].map(lambda x: x.replace(' (G)','')), co2_sueat['Food commodity ITEM'])
#warning, the matching from the wfp item "chocolate" and the cfp "chocolate" is spurious, must be avoided
co2_sueat['Food commodity ITEM']= np.where(np.logical_and(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].eq('CHOCOLATE')), 'CHOCOLATE ICE CREAM', co2_sueat['Food commodity ITEM'])
#for coherence sake i will manage in the same way also vanilla ice cream
co2_sueat['Food commodity ITEM']= np.where(np.logical_and(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].eq('VANILLA')), 'VANILLA ICE CREAM', co2_sueat['Food commodity ITEM'])
wfp_sueat['Food commodity ITEM'] = np.where(True, wfp_sueat['Food commodity ITEM'].map(lambda x: x.strip()), wfp_sueat['Food commodity ITEM'])
wfp_sueat['Food commodity ITEM'] = np.where(True, wfp_sueat['Food commodity ITEM'].map(lambda x: x.replace('*','')), wfp_sueat['Food commodity ITEM'])
wfp_sueat['Food commodity ITEM'] = np.where(True, wfp_sueat['Food commodity ITEM'].map(lambda x: x.replace(' (I)','')), wfp_sueat['Food commodity ITEM'])

In [ ]:
updated = pd.merge(co2_sueat, wfp_sueat, how='left', on = 'Food commodity ITEM', suffixes=('', '_new'))
updated['final_wfp'] = np.where(pd.notnull(updated['final_wfp_new']), updated['final_wfp_new'], updated['final_wfp'])
columns = ['FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
co2_sueat = updated[columns]
#co2_sueat[pd.notnull(co2_sueat['final_wfp'])]
#153 updated

###Step 3.1:

Matching Subtypology

Must remove parentheses and their contents, * and withespace both sides, must remove also some specific variation of the same string cfp side and wfp side.

In [ ]:
wfp_subtyp = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp_median_sub_typology.csv", sep= ';')

In [ ]:
prt_regex = '\((\w*\W*\s*)*\)' #remove parentheses and their content
star_regex = '\*' #remove *
and_space_regex = '\&\s' #remove '& ' in the string 'ROOTS & BULB'
comma_regex = '\,' #remove ',' in the string 'ROOTS, BULB'
other_regex = 'OTHER' #remove 'OTHER' in the string 'FRUIT OTHER'

wfp_subtyp = wfp_subtyp.rename(columns={"FOOD COMMODITY sub-TYPOLOGY": "Food commodity sub-TYPOLOGY"})
wfp_subtyp['Food commodity sub-TYPOLOGY'] = wfp_subtyp['Food commodity sub-TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True)
wfp_subtyp['Food commodity sub-TYPOLOGY'] = wfp_subtyp['Food commodity sub-TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True)
wfp_subtyp['Food commodity sub-TYPOLOGY'] = wfp_subtyp['Food commodity sub-TYPOLOGY'].replace(to_replace=comma_regex, value='', regex=True)
wfp_subtyp['Food commodity sub-TYPOLOGY'] = np.where(True, wfp_subtyp['Food commodity sub-TYPOLOGY'].map(lambda x: x.strip()), wfp_subtyp['Food commodity sub-TYPOLOGY'])
########################################
co2_sueat['Food commodity sub-TYPOLOGY'] = np.where(True, co2_sueat['Food commodity sub-TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True), co2_sueat['Food commodity sub-TYPOLOGY'] )
co2_sueat['Food commodity sub-TYPOLOGY'] = np.where(True, co2_sueat['Food commodity sub-TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True), co2_sueat['Food commodity sub-TYPOLOGY'] )
co2_sueat['Food commodity sub-TYPOLOGY'] = np.where(True, co2_sueat['Food commodity sub-TYPOLOGY'].replace(to_replace=and_space_regex, value='', regex=True), co2_sueat['Food commodity sub-TYPOLOGY'] )
co2_sueat['Food commodity sub-TYPOLOGY'] = np.where(True, co2_sueat['Food commodity sub-TYPOLOGY'].replace(to_replace=other_regex, value='', regex=True), co2_sueat['Food commodity sub-TYPOLOGY'] )
co2_sueat['Food commodity sub-TYPOLOGY'] = np.where(True, co2_sueat['Food commodity sub-TYPOLOGY'].map(lambda x: str(x).strip()), co2_sueat['Food commodity sub-TYPOLOGY'] )

print(wfp_subtyp['Food commodity sub-TYPOLOGY'].unique())
print(co2_sueat['Food commodity sub-TYPOLOGY'].unique())

['CITRUS FRUIT' 'FRUIT' 'OLIVES' 'WILD BERRIES' 'ASPARAGUS'
 'GREEN LEAFY VEGETABLES' 'ROOTS BULBS' 'VEGETABLES' 'CRUSTACEAN']
['-' 'nan' 'CITRUS FRUIT' 'FRUIT' 'OLIVES' 'WILD BERRIES' 'ASPARAGUS'
 'GREEN LEAFY VEGETABLES' 'ROOTS BULBS' 'VEGETABLES' 'BIVALVES'
 'CEPHALOPOD' 'CRUSTACEAN']


In [ ]:
#co2_sueat[pd.notnull(co2_sueat['final_wfp'])]

In [ ]:
updated = pd.merge(co2_sueat, wfp_subtyp, how='left', on='Food commodity sub-TYPOLOGY', suffixes=('', '_new'))
updated['final_wfp'] = np.where(np.logical_and(pd.isnull(updated['final_wfp']), pd.notnull(updated['Food commodity sub-TYPOLOGY'])), updated['median'], updated['final_wfp'])
columns = ['FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
co2_sueat = updated[columns]
#co2_sueat[pd.notnull(co2_sueat['final_wfp'])]
#167 updates

###Step 3.2:

Matching Typology

Must remove parentheses and their contents, * and withespace both sides.

In [ ]:
wfp_typ = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp_median_typology.csv", sep= ';')

In [ ]:
prt_regex = '\((\w*\W*\s*)*\)' #remove parentheses and their content
star_regex = '\*' #remove *
wfp_typ = wfp_typ.rename(columns={"FOOD COMMODITY TYPOLOGY": "Food commodity TYPOLOGY"})
wfp_typ['Food commodity TYPOLOGY'] = wfp_typ['Food commodity TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True)
wfp_typ['Food commodity TYPOLOGY'] = wfp_typ['Food commodity TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True)
wfp_typ['Food commodity TYPOLOGY'] = np.where(True, wfp_typ['Food commodity TYPOLOGY'].map(lambda x: x.strip()), wfp_typ['Food commodity TYPOLOGY'])
########################################
co2_sueat['Food commodity TYPOLOGY'] = np.where(True, co2_sueat['Food commodity TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True), co2_sueat['Food commodity TYPOLOGY'] )
co2_sueat['Food commodity TYPOLOGY'] = np.where(True, co2_sueat['Food commodity TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True), co2_sueat['Food commodity TYPOLOGY'] )
co2_sueat['Food commodity TYPOLOGY'] = np.where(True, co2_sueat['Food commodity TYPOLOGY'].map(lambda x: str(x).strip()), co2_sueat['Food commodity TYPOLOGY'] )

#print(wfp_typ['Food commodity TYPOLOGY'].unique())
#print(co2_sueat['Food commodity TYPOLOGY'].unique())

In [ ]:
a = set(co2_sueat['Food commodity TYPOLOGY'].unique())
b = set(wfp_typ['Food commodity TYPOLOGY'].unique())
print(a)
print(b)
print(a-b)
c = a.intersection(b)
print(len(c))
#the intersection is quite consistent, so I will use the matching that are already available between the two typology sets.

{'TOMATO PUREE,PEELED, CHOPPED', 'SEEDS', 'VEGETAL PROTEINS', 'POULTRY BONE FREE MEAT', 'LEGUMES', 'TOMATO SAUCE', 'FRUIT OPENFIELD', 'CHEESE HARD & SEMI-HARD', 'SWEETS', 'POULTRY MEAT WITH BONE', 'COFFEE GROUND & PARCHMENT', 'ICE CREAM', 'OIL', 'NUTS COVERED WITH CHOCOLATE', 'YOGURT', 'BOTTLED WATER', 'YEAST LIQUID', 'FRUIT IMPORTED', 'VEGETABLES HEATED GREENHOUSE', 'BEEF MEAT WITH BONE', 'VEGETABLES CANNED', 'EGG PASTA', 'KANGAROO MEAT', 'SNAILS', 'FISH', 'FISH PROCESSED', 'WINE', 'LEGUMES NOT HEATED GREENHOUSE', 'LEGUME FLOUR', 'MILK', 'NUTS PASTE', 'PASTA', 'COFFEE GREEN', 'YEAST COMPRESSED', 'VEGETAL CREAM', 'VEGETAL YOGURT', 'RABBIT MEAT WITH BONE', 'YEAST DRIED', 'BEER', 'LAMB MEAT WITH BONE', 'PORK CURED MEAT', 'VEGETABLES FROZEN', 'BUTTER', 'FRUIT FROZEN', 'BEEF BONE FREE MEAT', 'BREAKFAST CEREALS', 'NUTS', 'PESTO', 'EGGS', 'CHOCOLATE', 'COFFEE LIQUID', 'MARGARINE', 'VEGETABLES NOT HEATED GREENHOUSE', 'GRAIN FLOUR', 'CRISPBREAD', 'HONEY', 'FRUIT NOT HEATED GREENHOUSE', 'PORK M

In [ ]:
updated = pd.merge(co2_sueat, wfp_typ, how='left', on='Food commodity TYPOLOGY', suffixes=('', '_new'))
updated['final_wfp'] = np.where(np.logical_and(pd.isnull(updated['final_wfp']), pd.notnull(updated['median'])), updated['median'], updated['final_wfp'])
columns = ['FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
co2_sueat = updated[columns]
#co2_sueat[pd.notnull(co2_sueat['final_wfp'])]
#251 updates

There are still many ingredients without a wfp (73), let's analyze them and understand if there are compatible typologies.

In [ ]:
#co2_sueat[pd.isnull(co2_sueat['final_wfp'])]

The are matches that are (probably) semantically compatible:

CHEESE HARD & SEMI-HARD | CHEESE

CHOCOLATE | COCOA DERIVATES

FISH FROZEN | FISH

FISH PROCESSED | FISH

FRUIT JUICE | JUICE

JUICE LOCAL | JUICE

FRUIT FROZEN | FRUIT

FRUIT HEATED GREENHOUSE | FRUIT

FRUIT IMPORTED | FRUIT

FRUIT NOT HEATED GREENHOUSE | FRUIT

FRUIT JUICE IMPORTED | JUICE
_______________________________

LEGUMES NOT HEATED GREENHOUSE | LEGUMES  FRESH

LEGUMES CANNED | LEGUMES  FRESH

LEGUMES FROZEN | LEGUMES  FRESH

LEGUMES | LEGUMES  FRESH

(all legumes without wfp are green beans that are fresh legumes)
_______________________________

NUTS | NUTS SHELLED (since the shelled product is more common for final user)

SHELLFISH FROZEN | SHELLFISH

TOMATO PUREE,PEELED, CHOPPED (problems with spaces and commas)

VEGETABLES HEATED GREENHOUSE | VEGETABLES


In [ ]:
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='CHEESE HARD & SEMI-HARD', value='CHEESE', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='CHOCOLATE', value='COCOA DERIVATES', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='FISH FROZEN', value='FISH', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='FISH PROCESSED', value='FISH', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='FRUIT JUICE', value='JUICE', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='FRUIT FROZEN', value='FRUIT', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='FRUIT HEATED GREENHOUSE', value='FRUIT', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='FRUIT NOT HEATED GREENHOUSE', value='FRUIT', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='FRUIT IMPORTED', value='FRUIT', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='FRUIT JUICE IMPORTED', value='JUICE', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='JUICE LOCAL', value='JUICE', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(np.logical_and(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].eq('GREEN BEAN (fresh)')), 'LEGUMES  FRESH', co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='LEGUMES NOT HEATED GREENHOUSE', value='LEGUMES  FRESH', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='LEGUMES CANNED', value='LEGUMES  FRESH', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='LEGUMES FROZEN', value='LEGUMES  FRESH', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='NUTS', value='NUTS SHELLED', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='SHELLFISH FROZEN', value='SHELLFISH', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='TOMATO PUREE,PEELED, CHOPPED', value='TOMATO PUREE, PEELED, CHOPPED', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='VEGETABLES HEATED GREENHOUSE', value='VEGETABLES', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity TYPOLOGY'].replace(to_replace='VEGETABLES NOT HEATED GREENHOUSE', value='VEGETABLES', regex=True), co2_sueat['Food commodity TYPOLOGY'])
co2_sueat['Food commodity TYPOLOGY'] = np.where(True, co2_sueat['Food commodity TYPOLOGY'].map(lambda x: str(x).strip()), co2_sueat['Food commodity TYPOLOGY'] )

In [ ]:
updated = pd.merge(co2_sueat, wfp_typ, how='left', on='Food commodity TYPOLOGY', suffixes=('', '_new'))
updated['final_wfp'] = np.where(np.logical_and(pd.isnull(updated['final_wfp']), pd.notnull(updated['median'])), updated['median'], updated['final_wfp'])
columns = ['FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
co2_sueat = updated[columns]
#co2_sueat[pd.notnull(co2_sueat['final_wfp'])]
#302 TOTAL updates

In [ ]:
co2_sueat[pd.notnull(co2_sueat['final_wfp'])]

,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_co2,final_wfp
0,AGRICULTURAL PROCESSED,BEER IN CAN,BEER,-,"0,69",155
1,AGRICULTURAL PROCESSED,BEER IN GLASS,BEER,-,"0,96",155
2,AGRICULTURAL PROCESSED,BEER MODULAR CAN,BEER,-,"0,53",155
3,AGRICULTURAL PROCESSED,CHOCOLATE OR CREAM FILLED COOKIES,BISCUITS,-,"1,53",2902
4,AGRICULTURAL PROCESSED,SIMPLE COOKIES,BISCUITS,-,"1,39",1723
...,...,...,...,...,...,...
319,FISHING,OCTOPUS,SHELLFISH,CEPHALOPOD,"5,80",1856
320,FISHING,SQUID,SHELLFISH,CEPHALOPOD,"5,80",1856
321,FISHING,PRAWNS/SHRIMPS,SHELLFISH,CRUSTACEAN,"7,04","1856,0"
322,FISHING,OCTOPUS (F),SHELLFISH,-,"11,47",1856


In [ ]:
#co2_sueat[pd.isnull(co2_sueat['final_wfp'])]
#there are 22 items that are not mapped to a water footprint, we cant say anything about these items.

###Step 4:

Saving results in a new file restoring the original name of items and typologies.

In [ ]:
#co2_sueat

In [ ]:
co2_sueat_fin = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2_filtered.csv", sep= ';')
#co2_sueat_fin

In [ ]:
fin = pd.merge(co2_sueat, co2_sueat_fin, left_index=True, right_index=True)
columns = ['FOOD COMMODITY GROUP_x','Food commodity ITEM_y','Food commodity TYPOLOGY_y','Food commodity sub-TYPOLOGY_y','final_co2_x','final_wfp']
co2_sueat_save = fin[columns]
co2_sueat_save = co2_sueat_save.rename(columns={"FOOD COMMODITY GROUP_x": "FOOD COMMODITY GROUP","Food commodity ITEM_y": "Food commodity ITEM", "Food commodity TYPOLOGY_y":"Food commodity TYPOLOGY", "Food commodity sub-TYPOLOGY_y": "Food commodity sub-TYPOLOGY", "final_co2_x": "final_co2"})
co2_sueat_save

,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_co2,final_wfp
0,AGRICULTURAL PROCESSED,BEER IN CAN,BEER,-,"0,69",155
1,AGRICULTURAL PROCESSED,BEER IN GLASS,BEER,-,"0,96",155
2,AGRICULTURAL PROCESSED,BEER MODULAR CAN,BEER,-,"0,53",155
3,AGRICULTURAL PROCESSED,CHOCOLATE OR CREAM FILLED COOKIES**,BISCUITS,-,"1,53",2902
4,AGRICULTURAL PROCESSED,SIMPLE COOKIES**,BISCUITS,-,"1,39",1723
...,...,...,...,...,...,...
319,FISHING,OCTOPUS,SHELLFISH,CEPHALOPOD (SUB-TYP),"5,80",1856
320,FISHING,SQUID,SHELLFISH,CEPHALOPOD (SUB-TYP),"5,80",1856
321,FISHING,PRAWNS/SHRIMPS,SHELLFISH,CRUSTACEAN* (SUB-TYP) (Prawns/Shrimps),"7,04","1856,0"
322,FISHING,OCTOPUS (F),SHELLFISH FROZEN,-,"11,47",1856


In [ ]:
#saving
co2_sueat_save.to_csv('/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/_Finalized/cfp_wfp_ingredients.csv', sep= ';')

###Step 5(new): Integration of wfp sheet ingredient taking the relative wfp value and the cfp from the typology.

In [ ]:
co2_sueat = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2_filtered.csv", sep= ';')
wfp_sueat = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp_filtered.csv", sep= ';')
co2_sueat['final_wfp'] = None
co2_sueat['Food commodity ITEM'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].map(lambda x: x.strip()), co2_sueat['Food commodity ITEM'])
co2_sueat['Food commodity ITEM'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].map(lambda x: x.replace('*','')), co2_sueat['Food commodity ITEM'])
co2_sueat['Food commodity ITEM'] = np.where(pd.isnull(co2_sueat['final_wfp']), co2_sueat['Food commodity ITEM'].map(lambda x: x.replace(' (I)','')), co2_sueat['Food commodity ITEM'])
wfp_sueat['Food commodity ITEM'] = np.where(True, wfp_sueat['Food commodity ITEM'].map(lambda x: x.strip()), wfp_sueat['Food commodity ITEM'])
wfp_sueat['Food commodity ITEM'] = np.where(True, wfp_sueat['Food commodity ITEM'].map(lambda x: x.replace('*','')), wfp_sueat['Food commodity ITEM'])
wfp_sueat['Food commodity ITEM'] = np.where(True, wfp_sueat['Food commodity ITEM'].map(lambda x: x.replace(' (I)','')), wfp_sueat['Food commodity ITEM'])
wfp_sueat['final_co2'] = None
wfp_sueat

,Unnamed: 0,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_wfp,final_co2
0,0,AGRICULTURAL PROCESSED,APPLE JUICE,JUICE,-,1140,None
1,1,AGRICULTURAL PROCESSED,BARLEY FLAKES,BREAKFAST CEREALS,-,1977,None
2,2,AGRICULTURAL PROCESSED,BEANS (F),VEGETABLES FROZEN,-,801,None
3,3,AGRICULTURAL PROCESSED,BEER,BEER*,-,155,None
4,4,AGRICULTURAL PROCESSED,BEET SUGAR,SUGAR,-,865,None
...,...,...,...,...,...,...,...
315,315,FISHING,STURGEON,FISH,-,13420,None
316,316,FISHING,TILAPIA,FISH,-,2263,None
317,317,FISHING,TROUT,FISH,-,2188,None
318,318,FISHING,TURBOT,FISH,-,1504,None


In [ ]:
updated = pd.merge(wfp_sueat, co2_sueat, how='left', on = 'Food commodity ITEM', suffixes=('', '_new'))
updated
updated = updated[pd.isnull(updated['final_co2_new'])]
columns = ['Unnamed: 0','FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
wfp_sueat = updated[columns]
#181 elements
wfp_sueat

,Unnamed: 0,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_co2,final_wfp
1,1,AGRICULTURAL PROCESSED,BARLEY FLAKES,BREAKFAST CEREALS,-,None,1977
3,3,AGRICULTURAL PROCESSED,BEER,BEER*,-,None,155
5,5,AGRICULTURAL PROCESSED,BREAD,BREAD,-,None,667
10,10,AGRICULTURAL PROCESSED,CANE MOLASSE,SUGAR,-,None,527
12,12,AGRICULTURAL PROCESSED,CASSAVA FLOUR,STARCHY ROOT FLOUR,-,None,1878
...,...,...,...,...,...,...,...
317,313,FISHING,SEABASS,FISH,-,None,6524
318,314,FISHING,SILVERBARB,FISH,-,None,"2861,5"
319,315,FISHING,STURGEON,FISH,-,None,13420
320,316,FISHING,TILAPIA,FISH,-,None,2263


### 5.1 Matching subtypology

In [ ]:
co2_subtyp = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2_median_sub_typology.csv", sep= ';')
prt_regex = '\((\w*\W*\s*)*\)' #remove parentheses and their content
star_regex = '\*' #remove *

co2_subtyp = co2_subtyp.rename(columns={"FOOD COMMODITY sub-TYPOLOGY": "Food commodity sub-TYPOLOGY"})
co2_subtyp['Food commodity sub-TYPOLOGY'] = co2_subtyp['Food commodity sub-TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True)
co2_subtyp['Food commodity sub-TYPOLOGY'] = co2_subtyp['Food commodity sub-TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True)
co2_subtyp['Food commodity sub-TYPOLOGY'] = np.where(True, co2_subtyp['Food commodity sub-TYPOLOGY'].map(lambda x: x.strip()), co2_subtyp['Food commodity sub-TYPOLOGY'])
########################################
wfp_sueat['Food commodity sub-TYPOLOGY'] = np.where(True, wfp_sueat['Food commodity sub-TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True), wfp_sueat['Food commodity sub-TYPOLOGY'] )
wfp_sueat['Food commodity sub-TYPOLOGY'] = np.where(True, wfp_sueat['Food commodity sub-TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True), wfp_sueat['Food commodity sub-TYPOLOGY'] )
wfp_sueat['Food commodity sub-TYPOLOGY'] = np.where(True, wfp_sueat['Food commodity sub-TYPOLOGY'].map(lambda x: str(x).strip()), wfp_sueat['Food commodity sub-TYPOLOGY'] )

#print(wfp_subtyp['Food commodity sub-TYPOLOGY'].unique())
#print(co2_sueat['Food commodity sub-TYPOLOGY'].unique())

a = set(co2_subtyp['Food commodity sub-TYPOLOGY'].unique())
b = set(wfp_sueat['Food commodity sub-TYPOLOGY'].unique())
print(a)
print(b)

{'CITRUS FRUIT', 'CEPHALOPOD', 'BIVALVES', 'FRUIT', 'WILD BERRIES', 'ROOTS & BULBS', 'ASPARAGUS', 'OLIVES', 'CRUSTACEAN', 'VEGETABLES', 'GREEN LEAFY VEGETABLES'}
{'CITRUS FRUIT', 'nan', 'FRUIT', 'WILD BERRIES', 'ROOTS & BULBS', 'CRUSTACEAN', 'VEGETABLES', '-'}


In [ ]:
updated = pd.merge(wfp_sueat, co2_subtyp, how='left', on='Food commodity sub-TYPOLOGY', suffixes=('', '_new'))
updated
updated['final_co2'] = np.where(np.logical_and(pd.isnull(updated['final_co2']), pd.notnull(updated['Food commodity sub-TYPOLOGY'])), updated['median'], updated['final_co2'])
columns = ['Unnamed: 0','FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
wfp_sueat = updated[columns]
wfp_sueat


,Unnamed: 0,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_co2,final_wfp
0,1,AGRICULTURAL PROCESSED,BARLEY FLAKES,BREAKFAST CEREALS,-,NaN,1977
1,3,AGRICULTURAL PROCESSED,BEER,BEER*,-,NaN,155
2,5,AGRICULTURAL PROCESSED,BREAD,BREAD,-,NaN,667
3,10,AGRICULTURAL PROCESSED,CANE MOLASSE,SUGAR,-,NaN,527
4,12,AGRICULTURAL PROCESSED,CASSAVA FLOUR,STARCHY ROOT FLOUR,-,NaN,1878
...,...,...,...,...,...,...,...
176,313,FISHING,SEABASS,FISH,-,NaN,6524
177,314,FISHING,SILVERBARB,FISH,-,NaN,"2861,5"
178,315,FISHING,STURGEON,FISH,-,NaN,13420
179,316,FISHING,TILAPIA,FISH,-,NaN,2263


###5.2 Matching Typology

In [ ]:
co2_typ = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2_median_typology.csv", sep= ';')

prt_regex = '\((\w*\W*\s*)*\)' #remove parentheses and their content
star_regex = '\*' #remove *
co2_typ = co2_typ.rename(columns={"FOOD COMMODITY TYPOLOGY": "Food commodity TYPOLOGY"})
co2_typ['Food commodity TYPOLOGY'] = co2_typ['Food commodity TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True)
co2_typ['Food commodity TYPOLOGY'] = co2_typ['Food commodity TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True)
co2_typ['Food commodity TYPOLOGY'] = np.where(True, co2_typ['Food commodity TYPOLOGY'].map(lambda x: x.strip()), co2_typ['Food commodity TYPOLOGY'])
########################################
wfp_sueat['Food commodity TYPOLOGY'] = np.where(True, wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True), wfp_sueat['Food commodity TYPOLOGY'] )
wfp_sueat['Food commodity TYPOLOGY'] = np.where(True, wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True), wfp_sueat['Food commodity TYPOLOGY'] )
wfp_sueat['Food commodity TYPOLOGY'] = np.where(True, wfp_sueat['Food commodity TYPOLOGY'].map(lambda x: str(x).strip()), wfp_sueat['Food commodity TYPOLOGY'] )

print(co2_typ['Food commodity TYPOLOGY'].unique())
print(wfp_sueat['Food commodity TYPOLOGY'].unique())

['BEER' 'BISCUITS' 'BOTTLED WATER' 'BREAD FROZEN' 'BREAD'
 'BREAKFAST CEREALS' 'CHOCOLATE' 'COFFEE GROUND & PARCHMENT'
 'COFFEE LIQUID' 'CRACKERS' 'CRISPBREAD' 'DRESSING' 'EGG PASTA'
 'FRUIT FROZEN' 'FRUIT JUICE' 'FRUIT JUICE LOCAL' 'GRAIN FLOUR'
 'ICE CREAM' 'LEGUME FLOUR' 'LEGUMES CANNED' 'LEGUMES FROZEN' 'MARGARINE'
 'NUTS COVERED WITH CHOCOLATE' 'NUTS PASTE' 'OIL' 'PASTA' 'PESTO SAUCE'
 'SUGAR' 'SWEETS' 'TOMATO PUREE,PEELED, CHOPPED' 'TOMATO SAUCE'
 'VEGETABLES CANNED' 'VEGETABLES FROZEN' 'VEGETAL CREAM' 'VEGETAL MILK'
 'VEGETAL PROTEINS' 'VEGETAL YOGURT' 'WINE' 'YEAST COMPRESSED'
 'YEAST DRIED' 'YEAST LIQUID' 'BEEF BONE FREE MEAT' 'BEEF MEAT WITH BONE'
 'BUFFALO BONE FREE MEAT' 'BUTTER' 'CHEESE FRESH'
 'CHEESE HARD & SEMI-HARD' 'EDIBLE INSECTS' 'EGGS' 'HONEY' 'KANGAROO'
 'LAMB BONE FREE MEAT' 'LAMB MEAT WITH BONE' 'MILK' 'PORK BONE FREE MEAT'
 'PORK CURED MEAT' 'PORK MEAT WITH BONE' 'POULTRY BONE FREE MEAT'
 'POULTRY MEAT WITH BONE' 'RABBIT MEAT WITH BONE' 'SNAILS' 'YOGURT'
 'COFF

In [ ]:
updated = pd.merge(wfp_sueat, co2_typ, how='left', on='Food commodity TYPOLOGY', suffixes=('', '_new'))
updated
updated['final_co2'] = np.where(np.logical_and(pd.isnull(updated['final_co2']), pd.notnull(updated['median'])), updated['median'], updated['final_co2'])
columns = ['Unnamed: 0','FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
wfp_sueat = updated[columns]
wfp_sueat[pd.notnull(wfp_sueat['final_co2'])]

There are still many ingredients without a wfp (71), let's analyze them and understand if there are compatible typologies.

In [ ]:
wfp_sueat[pd.isnull(wfp_sueat['final_co2'])]

,Unnamed: 0,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_co2,final_wfp
4,12,AGRICULTURAL PROCESSED,CASSAVA FLOUR,STARCHY ROOT FLOUR,-,NaN,1878
5,15,AGRICULTURAL PROCESSED,COCOA BUTTER,COCOA DERIVATES,-,NaN,33938
6,17,AGRICULTURAL PROCESSED,COCOA PASTE,COCOA DERIVATES,-,NaN,24238
7,18,AGRICULTURAL PROCESSED,COCOA POWDER,COCOA DERIVATES,-,NaN,15618
14,27,AGRICULTURAL PROCESSED,DRIED CASSAVA,STARCHY TUBER,-,NaN,1610
...,...,...,...,...,...,...,...
156,282,CROPS,SUGAR BEET,SUGAR CROPS,-,NaN,132
157,283,CROPS,SUGAR CANE,SUGAR CROPS,-,NaN,210
159,289,CROPS,TOMATO DRIED,VEGETABLES DRIED,-,NaN,4276
161,292,CROPS,WALNUT SHELLED,NUTS SHELLED,-,NaN,"9172,5"


The are matches that are (probably) semantically compatible:

 COCOA DERIVATES | CHOCOLATE

 STARCHY TUBER | STARCHY TUBERS

 JUICE | FRUIT JUICE

 PROCESSED GRAINS | GRAINS

 CHEESE | CHEESE HARD & SEMI-HARD

 EGGS & DERIVATES | EGGS

 NUTS SHELLED | NUTS

 NUTS WITH SHELL | NUTS

 LEGUMES FRESH | LEGUMES

 LEGUMES DRIED | LEGUMES

 COFFEE | COFFEE GROUND & PARCHMENT

In [ ]:
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='COCOA DERIVATES', value='CHOCOLATE', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='STARCHY TUBER', value='STARCHY TUBERS', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='JUICE', value='FRUIT JUICE', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='PROCESSED GRAINS', value='GRAINS', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='CHEESE', value='CHEESE HARD & SEMI-HARD', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='EGGS & DERIVATES', value='EGGS', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='NUTS SHELLED', value='NUTS', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='NUTS WITH SHELL', value='NUTS', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='LEGUMES FRESH', value='LEGUMES', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='LEGUMES DRIED', value='LEGUMES', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(pd.isnull(wfp_sueat['final_co2']), wfp_sueat['Food commodity TYPOLOGY'].replace(to_replace='COFFEE', value='COFFEE GROUND & PARCHMENT', regex=True), wfp_sueat['Food commodity TYPOLOGY'])
wfp_sueat['Food commodity TYPOLOGY'] = np.where(True, wfp_sueat['Food commodity TYPOLOGY'].map(lambda x: str(x).strip()), wfp_sueat['Food commodity TYPOLOGY'] )

In [ ]:
updated = pd.merge(wfp_sueat, co2_typ, how='left', on='Food commodity TYPOLOGY', suffixes=('', '_new'))
updated
updated['final_co2'] = np.where(np.logical_and(pd.isnull(updated['final_co2']), pd.notnull(updated['median'])), updated['median'], updated['final_co2'])
columns = ['Unnamed: 0','FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
wfp_sueat = updated[columns]
wfp_sueat[pd.notnull(wfp_sueat['final_co2'])]

,Unnamed: 0,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_co2,final_wfp
0,1,AGRICULTURAL PROCESSED,BARLEY FLAKES,BREAKFAST CEREALS,-,"2,64",1977
1,3,AGRICULTURAL PROCESSED,BEER,BEER,-,"0,69",155
2,5,AGRICULTURAL PROCESSED,BREAD,BREAD,-,"0,86",667
3,10,AGRICULTURAL PROCESSED,CANE MOLASSE,SUGAR,-,"0,78",527
5,15,AGRICULTURAL PROCESSED,COCOA BUTTER,CHOCOLATE,-,"3,16",33938
...,...,...,...,...,...,...,...
176,313,FISHING,SEABASS,FISH,-,"5,19",6524
177,314,FISHING,SILVERBARB,FISH,-,"5,19","2861,5"
178,315,FISHING,STURGEON,FISH,-,"5,19",13420
179,316,FISHING,TILAPIA,FISH,-,"5,19",2263


For this last elements I haven't any compatible cfp typology, will be removed from the final dataset.

In [ ]:
wfp_sueat

,Unnamed: 0,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_co2,final_wfp
0,1,AGRICULTURAL PROCESSED,BARLEY FLAKES,BREAKFAST CEREALS,-,"2,64",1977
1,3,AGRICULTURAL PROCESSED,BEER,BEER,-,"0,69",155
2,5,AGRICULTURAL PROCESSED,BREAD,BREAD,-,"0,86",667
3,10,AGRICULTURAL PROCESSED,CANE MOLASSE,SUGAR,-,"0,78",527
4,12,AGRICULTURAL PROCESSED,CASSAVA FLOUR,STARCHY ROOT FLOUR,-,NaN,1878
...,...,...,...,...,...,...,...
176,313,FISHING,SEABASS,FISH,-,"5,19",6524
177,314,FISHING,SILVERBARB,FISH,-,"5,19","2861,5"
178,315,FISHING,STURGEON,FISH,-,"5,19",13420
179,316,FISHING,TILAPIA,FISH,-,"5,19",2263


###Step 6: 
Saving results in a new file restoring the original name of items and typologies.

In [ ]:
wfp_sueat_fin = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp_filtered.csv", sep= ';')
fin = pd.merge(wfp_sueat, wfp_sueat_fin, how='left', on='Unnamed: 0', suffixes=('', '_new'))
columns = ['FOOD COMMODITY GROUP','Food commodity ITEM_new','Food commodity TYPOLOGY_new','Food commodity sub-TYPOLOGY_new','final_co2','final_wfp']
wfp_sueat_save = fin[columns]
wfp_sueat_save = wfp_sueat_save.rename(columns={"Food commodity ITEM_new": "Food commodity ITEM", "Food commodity TYPOLOGY_new":"Food commodity TYPOLOGY", "Food commodity sub-TYPOLOGY_new": "Food commodity sub-TYPOLOGY", "final_co2_x": "final_co2"})
wfp_sueat_save

,FOOD COMMODITY GROUP,Food commodity ITEM,Food commodity TYPOLOGY,Food commodity sub-TYPOLOGY,final_co2,final_wfp
0,AGRICULTURAL PROCESSED,BARLEY FLAKES,BREAKFAST CEREALS,-,"2,64",1977
1,AGRICULTURAL PROCESSED,BEER*,BEER*,-,"0,69",155
2,AGRICULTURAL PROCESSED,BREAD,BREAD,-,"0,86",667
3,AGRICULTURAL PROCESSED,CANE MOLASSE,SUGAR,-,"0,78",527
4,AGRICULTURAL PROCESSED,CASSAVA FLOUR,STARCHY ROOT FLOUR,-,NaN,1878
...,...,...,...,...,...,...
176,FISHING,SEABASS,FISH,-,"5,19",6524
177,FISHING,SILVERBARB,FISH,-,"5,19","2861,5"
178,FISHING,STURGEON,FISH,-,"5,19",13420
179,FISHING,TILAPIA,FISH,-,"5,19",2263


### Step 7 appending the new dataframe to the previous dataset.

In [ ]:
#wfp_sueat[pd.isnull(wfp_sueat['final_co2'])]

In [ ]:
wfp_sueat_save = wfp_sueat_save[pd.notnull(wfp_sueat_save['final_co2'])]
wfp_sueat_save
sueatable = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/_Finalized/cfp_wfp_ingredients.csv",sep= ';')
sueatable = sueatable.append(wfp_sueat_save)
columns = ['FOOD COMMODITY GROUP','Food commodity ITEM','Food commodity TYPOLOGY','Food commodity sub-TYPOLOGY','final_co2','final_wfp']
sueatable = sueatable[columns]
sueatable = sueatable.sort_values(by=['FOOD COMMODITY GROUP','Food commodity ITEM'])
sueatable.to_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/_Finalized/cfp_wfp_ingredients.csv",sep= ';',index=False)
